# Calculation of image grid size

The google maps API takes a zoom value, the longitutude and lattitute. When making the image grid, we need to know by how much to change the longitude and lattitudes between each image so that they perfectly spaced and neither overlap nor have spaces in between.

The final calculation for a zoom level 19 and image size 640x640 gives:

Angle from E-W: 0.0034332275390625
Angle from N-S: 0.0017166137696733585


In [13]:
# from https://stackoverflow.com/questions/12507274/how-to-get-bounds-of-a-google-static-map

from __future__ import division
import math
MERCATOR_RANGE = 256

"""
Example of running the below function:
>>> import MercatorProjection
>>> centerLat = 49.141404
>>> centerLon = -121.960988
>>> zoom = 10
>>> mapWidth = 640
>>> mapHeight = 640
>>> centerPoint = MercatorProjection.G_LatLng(centerLat, centerLon)
>>> corners = MercatorProjection.getCorners(centerPoint, zoom, mapWidth, mapHeight)
>>> corners
{'E': -65.710988,
'N': 74.11120692972199,
'S': 0.333879313530149,
'W': -178.210988}
>>> mapURL = "http://maps.googleapis.com/maps/api/staticmap?center=%f,%f&zoom=%d&size=%dx%d&scale=2&maptype=roadmap&sensor=false"%(centerLat,centerLon,zoom,mapWidth,mapHeight)
>>> mapURL
http://maps.googleapis.com/maps/api/staticmap?center=49.141404,-121.960988&zoom=10&size=640x640&scale=2&maptype=roadmap&sensor=false'
"""

def  bound(value, opt_min, opt_max):
  if (opt_min != None): 
    value = max(value, opt_min)
  if (opt_max != None): 
    value = min(value, opt_max)
  return value


def  degreesToRadians(deg) :
  return deg * (math.pi / 180)


def  radiansToDegrees(rad) :
  return rad / (math.pi / 180)


class G_Point :
    def __init__(self,x=0, y=0):
        self.x = x
        self.y = y

class G_LatLng :
    def __init__(self,lt, ln):
        self.lat = lt
        self.lng = ln

class MercatorProjection :
    def __init__(self) :
      self.pixelOrigin_ =  G_Point( MERCATOR_RANGE / 2, MERCATOR_RANGE / 2)
      self.pixelsPerLonDegree_ = MERCATOR_RANGE / 360
      self.pixelsPerLonRadian_ = MERCATOR_RANGE / (2 * math.pi)


    def fromLatLngToPoint(self, latLng, opt_point=None) :
      point = opt_point if opt_point is not None else G_Point(0,0)
      origin = self.pixelOrigin_
      point.x = origin.x + latLng.lng * self.pixelsPerLonDegree_
      # NOTE(appleton): Truncating to 0.9999 effectively limits latitude to
      # 89.189.  This is about a third of a tile past the edge of the world tile.
      siny = bound(math.sin(degreesToRadians(latLng.lat)), -0.9999, 0.9999)
      point.y = origin.y + 0.5 * math.log((1 + siny) / (1 - siny)) * -     self.pixelsPerLonRadian_
      return point


    def fromPointToLatLng(self,point) :
          origin = self.pixelOrigin_
          lng = (point.x - origin.x) / self.pixelsPerLonDegree_
          latRadians = (point.y - origin.y) / -self.pixelsPerLonRadian_
          lat = radiansToDegrees(2 * math.atan(math.exp(latRadians)) - math.pi / 2)
          return G_LatLng(lat, lng)

#pixelCoordinate = worldCoordinate * pow(2,zoomLevel)

def getCorners(center, zoom, mapWidth, mapHeight):
    scale = 2**zoom
    proj = MercatorProjection()
    centerPx = proj.fromLatLngToPoint(center)
    SWPoint = G_Point(centerPx.x-(mapWidth/2)/scale, centerPx.y+(mapHeight/2)/scale)
    SWLatLon = proj.fromPointToLatLng(SWPoint)
    NEPoint = G_Point(centerPx.x+(mapWidth/2)/scale, centerPx.y-(mapHeight/2)/scale)
    NELatLon = proj.fromPointToLatLng(NEPoint)
    return {
        'N' : NELatLon.lat,
        'E' : NELatLon.lng,
        'S' : SWLatLon.lat,
        'W' : SWLatLon.lng,
    }

In [14]:
coords_start = (59.93717029103388, 10.807871254728301) # somewhere in Oslo, doesn't matter exactly where
zoom = 19
mapWidth =  640
mapHeight = 640
centerPoint = G_LatLng(coords_start[0], coords_start[1])

corners = getCorners(centerPoint, zoom, mapWidth, mapHeight)

angle_E_W = corners['E'] - corners['W']
angle_N_S = corners['N'] - corners['S']

print('Spacing required between images at zoom level 19 and image size 640x640:')
print(f'Angle from E-W {angle_E_W:.8f}')
print(f'Angle from N-S {angle_N_S:.8f}')

Spacing required between images at zoom level 19 and image size 640x640:
Angle from E-W 0.00171661
Angle from N-S 0.00085994


In [15]:
# get dimensions of images in m
# from  https://gis.stackexchange.com/questions/7430/what-ratio-scales-do-google-maps-zoom-levels-correspond-to
import math
lattitude = 10.7

metersPerPx = 156543.03392 * math.cos(lattitude * math.pi / 180) / math.pow(2, zoom)
print(f"{metersPerPx:.2f} metres per pixel.")
print(f"{metersPerPx*mapHeight:.2f} metres per image in N-S direction.")
print(f"{metersPerPx*mapWidth:.2f} metres per image in E-W direction.")

0.29 metres per pixel.
187.77 metres per image in N-S direction.
187.77 metres per image in E-W direction.
